In [1]:
#!pip install pulp numpy --quiet

import numpy as np
import pulp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 23.4 MB/s eta 0:00:00


In [2]:
class MovingAverageForecaster:
    """
    Simple k‑period moving‑average forecaster (scratch implementation).
    """

    def __init__(self, k: int = 3):
        if k < 1:
            raise ValueError("Window k must be ≥1.")
        self.k = k
        self._history = None          # type: np.ndarray | None
        self._latest_ma = None        # type: float | None

    def fit(self, series):
        """
        Store the series and compute the latest moving average.
        Returns the latest MA value.
        """
        y = np.asarray(series, dtype=float)
        if y.ndim != 1 or y.size < self.k:
            raise ValueError("Series must contain at least k observations.")
        self._history = y
        self._latest_ma = y[-self.k:].mean()
        return float(self._latest_ma)

    def forecast(self, steps: int = 1):
        """Return *steps* copies of the latest moving average."""
        if self._latest_ma is None:
            raise RuntimeError("Call .fit() first.")
        return [self._latest_ma] * steps


In [3]:
class TransportationOptimizer:
    """
    Min‑cost transportation linear program:
    Min  Σ c_ij x_ij
    s.t. Σ_j x_ij ≤ supply_i      for each source i
         Σ_i x_ij = demand_j      for each destination j
         x_ij ≥ 0
    """

    def __init__(self, supply, demand, unit_cost, solver: str | None = None):
        self.S = list(supply.keys())
        self.D = list(demand.keys())
        self.supply = supply
        self.demand = demand
        self.cost = unit_cost
        self.solver_name = solver
        self._prob = None           # type: pulp.LpProblem | None
        self._x = None              # decision‑var dict
        self._status = None         # "Optimal", "Infeasible", …

    # --------------------------------------------
    def _build_model(self):
        prob = pulp.LpProblem("Transportation", pulp.LpMinimize)
        x = pulp.LpVariable.dicts("ship", (self.S, self.D), lowBound=0)

        # Objective
        prob += pulp.lpSum(self.cost[i, j] * x[i][j] for i in self.S for j in self.D)

        # Supply constraints
        for i in self.S:
            prob += pulp.lpSum(x[i][j] for j in self.D) <= self.supply[i], f"Supply_{i}"

        # Demand constraints
        for j in self.D:
            prob += pulp.lpSum(x[i][j] for i in self.S) == self.demand[j], f"Demand_{j}"

        self._prob, self._x = prob, x

    # --------------------------------------------
    def solve(self, msg: bool = False):
        if self._prob is None:
            self._build_model()
        solver = (
            pulp.getSolver(self.solver_name)
            if self.solver_name
            else pulp.PULP_CBC_CMD(msg=msg)
        )
        self._prob.solve(solver)
        self._status = pulp.LpStatus[self._prob.status]
        return self._status

    # --------------------------------------------
    def total_cost(self):
        if self._status != "Optimal":
            raise ValueError("Model not optimal – no objective value.")
        return pulp.value(self._prob.objective)

    def shipment_plan(self):
        if self._status != "Optimal":
            raise ValueError("Model not optimal – no shipments.")
        return {
            (i, j): pulp.value(self._x[i][j])
            for i in self.S
            for j in self.D
            if pulp.value(self._x[i][j]) > 1e-6
        }


In [4]:
# ---- Forecasting example ----
hist_demand = [105, 98, 102, 100, 107, 103, 99]
ma = MovingAverageForecaster(k=3)
latest_ma = ma.fit(hist_demand)
print(f"Latest 3‑period moving average: {latest_ma:.2f}")
print("Forecast for next 2 periods:", ma.forecast(2))

# ---- Transportation LP example ----
supply  = {"PlantA": 120, "PlantB": 150}
demand  = {"North": 80, "South": 90, "East": 75, "West": 25}
costs   = {("PlantA","North"):4, ("PlantA","South"):6, ("PlantA","East"):9, ("PlantA","West"):7,
           ("PlantB","North"):5, ("PlantB","South"):4, ("PlantB","East"):7, ("PlantB","West"):3}

opt = TransportationOptimizer(supply, demand, costs)
print("LP status:", opt.solve())
print("Minimum cost:", opt.total_cost())
print("Shipments:")
for (src, dst), qty in opt.shipment_plan().items():
    print(f"  {src} → {dst} : {qty}")


Latest 3‑period moving average: 103.00
Forecast for next 2 periods: [np.float64(103.0), np.float64(103.0)]
LP status: Optimal
Minimum cost: 1360.0
Shipments:
  PlantA → North : 80.0
  PlantA → East : 40.0
  PlantB → South : 90.0
  PlantB → East : 35.0
  PlantB → West : 25.0


In [5]:
# Example
supply  = {"Plant1": 500, "Plant2": 650, "Plant3": 400}
demand  = {"ZoneA": 350, "ZoneB": 420, "ZoneC": 380, "ZoneD": 400}

#generate unit costs randomly for every (plant, zone) pair
rng = np.random.default_rng(42)
costs = {(i, j): rng.integers(3, 10) for i in supply for j in demand}

opt2 = TransportationOptimizer(supply, demand, costs)
opt2.solve()
print("Total cost:", opt2.total_cost())
print("First 10 shipments:", list(opt2.shipment_plan().items())[:10])


Total cost: 6220.0
First 10 shipments: [(('Plant1', 'ZoneA'), 350.0), (('Plant1', 'ZoneB'), 20.0), (('Plant1', 'ZoneD'), 130.0), (('Plant2', 'ZoneC'), 380.0), (('Plant2', 'ZoneD'), 270.0), (('Plant3', 'ZoneB'), 400.0)]


In [6]:
# Forecasting – uses a 6‑month window (k=6) on 2 years of monthly data, then predicts the next 3 months.
months = [
    "2023‑01","2023‑02","2023‑03","2023‑04","2023‑05","2023‑06",
    "2023‑07","2023‑08","2023‑09","2023‑10","2023‑11","2023‑12",
    "2024‑01","2024‑02","2024‑03","2024‑04","2024‑05","2024‑06",
    "2024‑07","2024‑08","2024‑09","2024‑10","2024‑11","2024‑12",
]
# synthetic monthly demand (units)
demand_hist = [
    990, 1005, 1012, 998, 1020, 1033,
    1041, 1050, 1066, 1052, 1070, 1082,
    1095, 1102, 1116, 1123, 1_37, 1149,
    1150, 1175, 1185, 1197, 1206, 1220
]

ma6 = MovingAverageForecaster(k=6)
latest = ma6.fit(demand_hist)
print("=== 6‑period Moving Average Forecast ===")
print(f"  Latest MA (Dec‑24): {latest:.1f} units")
print(f"  Forecast for Jan‑25 to Mar‑25:", ma6.forecast(3))
print()

# Optimisation – scales the LP up to 40 decision variables (5 plants × 8 markets) plus 13 constraints and solves with CBC.
plants  = {f"Plant{i}": cap for i, cap in zip("ABCDE", [600, 750, 500, 650, 550])}
markets = {
    "North": 320, "South": 280, "East": 300, "West": 260,
    "Central": 310, "N‑East": 230, "N‑West": 240, "S‑Central": 230
}

# Build a cost dict with reproducible random integers [3, 12]
rng = np.random.default_rng(2025)
unit_cost = {(p, m): int(rng.integers(3, 13)) for p in plants for m in markets}

opt_big = TransportationOptimizer(plants, markets, unit_cost)
status_big = opt_big.solve()
print("=== Transportation LP (5 × 8) ===")
print("  Solver status :", status_big)
print("  Minimum cost  :", opt_big.total_cost())
print("  shipment decisions:")
for (src, dst), qty in list(opt_big.shipment_plan().items())[:15]:
    print(f"    {src:6} → {dst:9} : {qty}")


=== 6‑period Moving Average Forecast ===
  Latest MA (Dec‑24): 1188.8 units
  Forecast for Jan‑25 to Mar‑25: [np.float64(1188.8333333333333), np.float64(1188.8333333333333), np.float64(1188.8333333333333)]

=== Transportation LP (5 × 8) ===
  Solver status : Optimal
  Minimum cost  : 9620.0
  shipment decisions:
    PlantA → North     : 10.0
    PlantB → East      : 260.0
    PlantB → West      : 260.0
    PlantB → N‑East    : 230.0
    PlantC → Central   : 310.0
    PlantC → S‑Central : 190.0
    PlantD → South     : 280.0
    PlantD → East      : 40.0
    PlantD → S‑Central : 40.0
    PlantE → North     : 310.0
    PlantE → N‑West    : 240.0
